# HPC Allocation

## Flux 

### Resource Assignment

In [1]:
from executorlib import Executor

In [2]:
%%time
with Executor(backend="flux", flux_executor_pmi_mode="pmix") as exe:
    future_lst = [exe.submit(sum, [i, i]) for i in range(2, 5)]
    print([f.result() for f in future_lst])

[4, 6, 8]
CPU times: user 44.4 ms, sys: 16.4 ms, total: 60.7 ms
Wall time: 1.09 s


In [3]:
def add_funct(a, b):
    return a + b

In [4]:
with Executor(backend="flux", flux_executor_pmi_mode="pmix") as exe:
    future = None
    for i in range(1, 4):
        if future is None:
            future = exe.submit(add_funct, i, i)
        else:
            future = exe.submit(add_funct, i, future)
    print(future.result())

7


In [5]:
def calc(i):
    from mpi4py import MPI

    size = MPI.COMM_WORLD.Get_size()
    rank = MPI.COMM_WORLD.Get_rank()
    return i, size, rank

In [6]:
with Executor(backend="flux", flux_executor_pmi_mode="pmix") as exe:
    fs = exe.submit(calc, 3, resource_dict={"cores": 2})
    print(fs.result())

[(3, 2, 0), (3, 2, 1)]


### Nested executors

In [7]:
import flux.resource

In [8]:
with flux.Flux() as handle:
    rs = flux.resource.status.ResourceStatusRPC(handle).get()
    rl = flux.resource.list.resource_list(handle).get()
    print(
        "nodes: ", rs.nodelist, " #cores: ", rl.all.ncores, " #free: ", rl.free.ncores
    )

nodes:  cmpc06  #cores:  4  #free:  4


### Resource Monitoring

In [9]:
! flux resource list

     STATE NNODES   NCORES    NGPUS NODELIST
      free      1        4        0 cmpc06
 allocated      0        0        0 
      down      0        0        0 


In [10]:
! flux jobs -a

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
   ƒ2oHWAuoD janssen  python     CD      2      1   0.864s cmpc06
   ƒ2nv5oHeF janssen  python     CD      1      1   0.601s cmpc06
   ƒ2ngP9vbR janssen  python     CD      1      1   0.515s cmpc06
   ƒ2nQ2KrEj janssen  python     CD      1      1   0.525s cmpc06
   ƒ2n6PNPnp janssen  python     CD      1      1   0.484s cmpc06
   ƒ2n6PNPno janssen  python     CD      1      1   0.477s cmpc06
   ƒ2n6PNPnq janssen  python     CD      1      1   0.472s cmpc06
   ƒ22V35h75 janssen  pysqa      CD      2      1   3.679s cmpc06
   ƒ21fZzgnj janssen  pysqa      CD      1      1   2.836s cmpc06
   ƒ21WSj7Bd janssen  pysqa      CD      1      1   2.916s cmpc06
   ƒ21LrmkWK janssen  pysqa      CD      1      1   2.870s cmpc06
   ƒ21BacFD9 janssen  pysqa      CD      1      1   2.922s cmpc06
    ƒzPbWXEo janssen  pysqa      CD      1      1   2.898s cmpc06
    ƒzFHgYej janssen  pysqa      CD      1      1   2.965s cmpc06
    ƒz6Lnt23

## SLURM 

### Resource Assignment

### Advanced Configuration
* explain additional arguments

### Combined with Flux

## Other Queuing Systems
While primarily Flux and SLURM are supported it should be possible to adopt the configuration to other queuing systems. 

In [11]:
from executorlib.standalone.interactive.spawner import generate_slurm_command

In [12]:
generate_slurm_command??

Signature:
generate_slurm_command(
    cores: int,
    cwd: str,
    threads_per_core: int = 1,
    gpus_per_core: int = 0,
    openmpi_oversubscribe: bool = False,
    slurm_cmd_args: list[str] = [],
) -> list[str]
Source:   
def generate_slurm_command(
    cores: int,
    cwd: str,
    threads_per_core: int = 1,
    gpus_per_core: int = 0,
    openmpi_oversubscribe: bool = False,
    slurm_cmd_args: list[str] = [],
) -> list[str]:
    """
    Generate the command list for the SLURM interface.

    Args:
        cores (int): The number of cores.
        cwd (str): The current working directory.
        threads_per_core (int, optional): The number of threads per core. Defaults to 1.
        gpus_per_core (int, optional): The number of GPUs per core. Defaults to 0.
        openmpi_oversubscribe (bool, optional): Whether to oversubscribe the cores. Defaults to False.
        slurm_cmd_args (list[str], optional): Additional command line arguments. Defaults to [].

    Returns:
        lis